# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# The PrettyTable library is used to display data in tabular form and include headings
# from the select query statements. Uncomment the pip install statement below if don't have
# PTable installed yet
# !pip install PTable

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [3]:
def get_filepaths_list():
    # Get your current folder and subfolder event data
    filepath = os.getcwd() + '/event_data'

    # Create a for loop to create a list of files and collect each filepath
    for root, dirs, files in os.walk(filepath):
        if root.endswith(".ipynb_checkpoints"):
            continue

        # join the file path and roots with the subdirectories using glob
        file_path_list = glob.glob(os.path.join(root,'*'))
    
    return file_path_list

file_path_list = get_filepaths_list()

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
def generate_event_datafile(file_path_list):
    # initiating an empty list of rows that will be generated from each file
    full_data_rows_list = [] 

    # for every filepath in the file path list 
    for f in file_path_list:

        # reading csv file 
        with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
            # creating a csv reader object 
            csvreader = csv.reader(csvfile) 
            next(csvreader)

             # extracting each data row one by one and append it        
            for line in csvreader:
                full_data_rows_list.append(line) 

    # creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
    # Apache Cassandra tables
    csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

    with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
        writer = csv.writer(f, dialect='myDialect')
        writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                    'level','location','sessionId','song','userId'])
        for row in full_data_rows_list:
            if (row[0] == ''):
                continue
            writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


generate_event_datafile(file_path_list)

In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Data Modelling and Loading data into Apache Cassandra Database tables. 

## Now ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data appears like in the <font color=red>**event_datafile_new.csv**</font> after the pre-processing above:<br>

<img src="images/image_event_datafile_new.jpg">

## Creating a cluster and session connection, keyspace and setting the keyspace of the Apache Cassandra instance in the following cells below

#### Creating a Cluster

In [6]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify
        WITH REPLICATION  = {'class': 'SimpleStrategy', 'replication_factor' : 1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### With Apache Cassandra you model the database tables on the queries you want to run. We need to create tables depending on the given queries.

### Creating queries following the three asked questions of the data

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


### Query 1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

```sql
SELECT artist, song, length FROM music_history WHERE session_id = 338 AND item_in_session = 4
```

- **The music_history will be partitioned by the session_id, and item_in_session will be the clustering column to make the primary key unique. Assuming a different item_in_session for a givem session_id**.

In [9]:
query = "CREATE TABLE IF NOT EXISTS music_history "
query += "(session_id int, item_in_session int, artist varchar, song varchar, length decimal, PRIMARY KEY (session_id, item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_history (session_id, item_in_session, artist, song, length)"
        query = query + " VALUES(%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### SELECT to verify that the data have been inserted into each table

In [11]:
query = "SELECT artist, song, length FROM music_history WHERE session_id = 338 AND item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

df = pd.DataFrame(rows._current_rows)
df

artist                             song    length
0  Faithless  Music Matters (Mark Knight Dub)  495.3073

### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
```sql
SELECT artist, song, item_in_session, username FROM user_artist_view WHERE user_id = 10 AND session_id = 182
```

- **The user_artist_view table will be partitioned by user_id and session_id and item_in_session will be used as the clustering column since we have songs sorted by itemInSession**.

In [12]:
query = "CREATE TABLE IF NOT EXISTS user_artist_view "
query += "(user_id int, session_id int, item_in_session int, artist varchar, username varchar, song varchar, PRIMARY KEY ((user_id, session_id), item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)


In [13]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_artist_view (user_id, session_id, item_in_session, artist, song, username)"
        query = query + " VALUES(%s, %s, %s, %s, %s, %s)"

        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], f"{line[1]} {line[4]}"))

In [14]:
query = "SELECT artist, song, item_in_session, username FROM user_artist_view WHERE user_id = 10 AND session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

table = PrettyTable(["Artists", "Song", "Item in Session", "User"])
for row in rows:
    table.add_row([row.artist, row.song, row.item_in_session, row.username])
print(table)

+-------------------+------------------------------------------------------+-----------------+-------------+
|      Artists      |                         Song                         | Item in Session |     User    |
+-------------------+------------------------------------------------------+-----------------+-------------+
|  Down To The Bone |                  Keep On Keepin' On                  |        0        | Sylvie Cruz |
|    Three Drives   |                     Greece 2000                      |        1        | Sylvie Cruz |
| Sebastien Tellier |                      Kilometer                       |        2        | Sylvie Cruz |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |        3        | Sylvie Cruz |
+-------------------+------------------------------------------------------+-----------------+-------------+


### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

```sql
SELECT username FROM music_users WHERE song = 'All Hands Against His Own'
```

- **The table music_users will be partitioned by song and user_id as the clustering column as I think the combination of the two is enough to make the primary key unique. Because a user can stream different kinds of songs.**

In [15]:
query = "CREATE TABLE IF NOT EXISTS music_users "
query += "(song varchar, user_id int, username varchar, PRIMARY KEY(song, user_id))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [16]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_users (song, user_id, username)"
        query = query + " VALUES(%s, %s, %s)"

        session.execute(query, (line[9], int(line[10]), f"{line[1]} {line[4]}"))

In [17]:
query = "SELECT username FROM music_users WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

table = PrettyTable(["Username"])
for row in rows:
    table.add_row([row.username])
print(table)

+------------------+
|     Username     |
+------------------+
| Jacqueline Lynch |
|   Tegan Levine   |
|   Sara Johnson   |
+------------------+


### Dropping the tables before closing out the sessions

In [18]:
query = "DROP TABLE IF EXISTS music_history"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [19]:
query = "DROP TABLE IF EXISTS user_artist_view"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [20]:
query = "DROP TABLE IF EXISTS music_users"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()